# Final Paper

##### Kimberly Liu & Isaac Tabor

# Project Results

**Clear Prediction Question:** What are the most likely 2025 March Madness outcomes according to our XGBoost model?



Start by cloning the repository and adjusting file path as necessary.

Ex:

`! git clone https://github.com/kimberlyyliuu/DS3001-Project`

`cd /content/DS3001-Project/data`

**Importing Data**

In [2]:
! git clone https://github.com/kimberlyyliuu/DS3001-Project

Cloning into 'DS3001-Project'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 223 (delta 84), reused 72 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (223/223), 21.94 MiB | 4.29 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Updating files: 100% (46/46), done.


In [3]:
cd /content/DS3001-Project/data

/content/DS3001-Project/data


In [4]:
import numpy as np
import pandas as pd
"""#Section 1 """
# MTeams_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MTeams.csv")
# cd /content/DS3001-Project/data
MTeams_df = pd.read_csv("MTeams.csv")

# MSeasons_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MSeasons.csv")
MSeasons_df = pd.read_csv("MSeasons.csv")

# MTourneySeeds_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MNCAATourneySeeds.csv")
# MTourneySeeds_df = pd.read_csv("/content/DS3001-Project/data/MNCAATourneySeeds.csv")

# MRegularSeasonCompactResults_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MRegularSeasonCompactResults.csv")
MRegularSeasonCompactResults_df = pd.read_csv("MRegularSeasonCompactResults.csv")

""" ## Section 2 """
# MRegularSeasonDetailedResults_df =  pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MRegularSeasonDetailedResults.csv")
MRegularSeasonDetailedResults_df =  pd.read_csv("MRegularSeasonDetailedResults.csv")

# MNCAATourneyDetailedResults_df =   pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MNCAATourneyDetailedResults.csv")
MNCAATourneyDetailedResults_df =   pd.read_csv("MNCAATourneyDetailedResults.csv")

# MTeamConferences_df =pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MTeamConferences.csv")
MTeamConferences_df =pd.read_csv("MTeamConferences.csv")

# MGameCities_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MGameCities.csv")
MGameCities_df = pd.read_csv("MGameCities.csv")

#MConferenceTourneyGames_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MConferenceTourneyGames.csv")
MConferenceTourneyGames_df = pd.read_csv("MConferenceTourneyGames.csv")

# df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MMasseyOrdinals.csv")
df = pd.read_csv("MMasseyOrdinals.csv")

# MNCAATourneySlots_df = pd.read_csv("/Users/kimberlyliu/Downloads/DS 3001/DS3001-Project/data/MNCAATourneySlots.csv")
MNCAATourneySlots_df = pd.read_csv("MNCAATourneySlots.csv")



In [5]:
"""## Drop Teams that have not been D1 since 2003 """
MTeams_df = MTeams_df[MTeams_df['LastD1Season'] >= 2003]

# Showing older games to newest games
MRegularSeasonDetailedResults_df.sort_values(by=['Season', 'DayNum'], ascending=[True, True])


""" Add Team Name for Win and Lose """
MRegularSeasonDetailedResults_df['WTeamName'] = MRegularSeasonDetailedResults_df['WTeamID'].map(MTeams_df.set_index('TeamID')['TeamName'])
MRegularSeasonDetailedResults_df['LTeamName'] = MRegularSeasonDetailedResults_df['LTeamID'].map(MTeams_df.set_index('TeamID')['TeamName'])

""" add game type and city id """
merged_df = MRegularSeasonDetailedResults_df.merge(
    MGameCities_df[['Season', 'DayNum', 'WTeamID', 'LTeamID', 'CRType', 'CityID']],
    on=['Season', 'DayNum', 'WTeamID', 'LTeamID'],
    how='left'
)

print(merged_df.info())
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117748 entries, 0 to 117747
Data columns (total 38 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Season     117748 non-null  int64  
 1   DayNum     117748 non-null  int64  
 2   WTeamID    117748 non-null  int64  
 3   WScore     117748 non-null  int64  
 4   LTeamID    117748 non-null  int64  
 5   LScore     117748 non-null  int64  
 6   WLoc       117748 non-null  object 
 7   NumOT      117748 non-null  int64  
 8   WFGM       117748 non-null  int64  
 9   WFGA       117748 non-null  int64  
 10  WFGM3      117748 non-null  int64  
 11  WFGA3      117748 non-null  int64  
 12  WFTM       117748 non-null  int64  
 13  WFTA       117748 non-null  int64  
 14  WOR        117748 non-null  int64  
 15  WDR        117748 non-null  int64  
 16  WAst       117748 non-null  int64  
 17  WTO        117748 non-null  int64  
 18  WStl       117748 non-null  int64  
 19  WBlk       117748 non-n

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDR,LAst,LTO,LStl,LBlk,LPF,WTeamName,LTeamName,CRType,CityID
0,2003,10,1104,68,1328,62,N,0,27,58,...,22,8,18,9,2,20,Alabama,Oklahoma,NaN,NaN
1,2003,10,1272,70,1393,63,N,0,26,62,...,25,7,12,8,6,16,Memphis,Syracuse,NaN,NaN
2,2003,11,1266,73,1437,61,N,0,24,58,...,22,9,12,2,5,23,Marquette,Villanova,NaN,NaN
3,2003,11,1296,56,1457,50,N,0,18,38,...,20,9,19,4,3,23,N Illinois,Winthrop,NaN,NaN
4,2003,11,1400,77,1208,71,N,0,30,61,...,15,12,10,7,1,14,Texas,Georgia,NaN,NaN


In [6]:
merged_df['CRType'].value_counts() # Note that we only want and have regular season stats here.

,count
CRType,
Regular,83674


In [7]:
# Add conferences of winning and losing teams.

""" Merge to bring in the conference abbreviation as ConfAbbrev """
merged_df = merged_df.merge(
    MTeamConferences_df[['Season', 'TeamID', 'ConfAbbrev']],
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID'],
    how='left'
)

""" Rename the imported column to WConf and drop the duplicate TeamID column from the merge"""
merged_df.rename(columns={'ConfAbbrev': 'WConf'}, inplace=True)
merged_df.drop(columns=['TeamID'], inplace=True)


""" Merge to bring in the conference abbreviation as ConfAbbrev"""
merged_df = merged_df.merge(
    MTeamConferences_df[['Season', 'TeamID', 'ConfAbbrev']],
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID'],
    how='left'
)

merged_df.rename(columns={'ConfAbbrev': 'LConf'}, inplace=True)
merged_df.drop(columns=['TeamID'], inplace=True)

print(len(merged_df))
merged_df.head()

117748


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LTO,LStl,LBlk,LPF,WTeamName,LTeamName,CRType,CityID,WConf,LConf
0,2003,10,1104,68,1328,62,N,0,27,58,...,18,9,2,20,Alabama,Oklahoma,NaN,NaN,sec,big_twelve
1,2003,10,1272,70,1393,63,N,0,26,62,...,12,8,6,16,Memphis,Syracuse,NaN,NaN,cusa,big_east
2,2003,11,1266,73,1437,61,N,0,24,58,...,12,2,5,23,Marquette,Villanova,NaN,NaN,cusa,big_east
3,2003,11,1296,56,1457,50,N,0,18,38,...,19,4,3,23,N Illinois,Winthrop,NaN,NaN,mac,big_south
4,2003,11,1400,77,1208,71,N,0,30,61,...,10,7,1,14,Texas,Georgia,NaN,NaN,big_twelve,sec


Above we have data at the game-level unit of observation.

Below, we aggregate the data to the season-level for each team.

First, we format the game-level data to a new unit of observation such that there is 1 row for each team's game win or loss. For example, before UVA versus Duke Feb 9, 2019 had one row. Now, it has 2 rows, one for UVA documenting the loss and one for Duke documenting the win.

Then, this data can be aggregated for season-level data for each team.

In [8]:
# Team Season Aggregations

df = merged_df.copy()  # Work on a copy to avoid modifying the original data

# Create a version where each row represents the winning team
df_wins = df.copy()
df_wins["TeamID"] = df_wins["WTeamID"]
df_wins["OpponentID"] = df_wins["LTeamID"]
df_wins["ScoreDiff"] = df_wins["WScore"] - df_wins["LScore"]  # Positive = win margin
df_wins["Win"] = 1  # Mark as a win
df_wins["GameResult"] = "W"

# Create a version where each row represents the losing team
df_losses = df.copy()
df_losses["TeamID"] = df_losses["LTeamID"]
df_losses["OpponentID"] = df_losses["WTeamID"]
df_losses["ScoreDiff"] = df_losses["LScore"] - df_losses["WScore"]  # Negative = loss margin
df_losses["Win"] = 0  # Mark as a loss
df_losses["GameResult"] = "L"

# Combine both win and loss versions into one dataset
df_team_games = pd.concat([df_wins, df_losses])

# Aggregate stats for each team per season
team_season_agg = (
    df_team_games.groupby(["Season", "TeamID"])
    .agg(
        AvgScoreDiff=("ScoreDiff", "mean"),    # Average point difference
        MedianScoreDiff=("ScoreDiff", "median"),
        MinScoreDiff=("ScoreDiff", "min"),
        MaxScoreDiff=("ScoreDiff", "max"),
        Wins=("Win", "sum"),                   # Total wins
        Losses=("GameResult", lambda x: (x == "L").sum()),  # Count losses
        WinPercentage=("Win", "mean"),         # Win rate
        AvgFGM=("WFGM", "mean"),               # Field goals made per game
        AvgFGA=("WFGA", "mean"),               # Field goals attempted per game
        AvgFGPerc=("WFGM", lambda x: x.sum() / x.count()),  # Shooting percentage
        Avg3PM=("WFGM3", "mean"),              # 3-pointers made per game
        Avg3PA=("WFGA3", "mean"),              # 3-pointers attempted per game
        AvgFTM=("WFTM", "mean"),               # Free throws made per game
        AvgFTA=("WFTA", "mean"),               # Free throws attempted per game
        AvgRebounds=("WOR", "mean"),           # Offensive rebounds
        AvgDefRebounds=("WDR", "mean"),        # Defensive rebounds
        AvgAssists=("WAst", "mean"),           # Assists
        AvgTurnovers=("WTO", "mean"),          # Turnovers
        AvgSteals=("WStl", "mean"),            # Steals
        AvgBlocks=("WBlk", "mean"),            # Blocks
        AvgFouls=("WPF", "mean"),              # Fouls committed
    )
    .reset_index()
)

team_season_agg.head() # Preview the aggregated data


,Season,TeamID,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,Wins,Losses,WinPercentage,AvgFGM,...,Avg3PA,AvgFTM,AvgFTA,AvgRebounds,AvgDefRebounds,AvgAssists,AvgTurnovers,AvgSteals,AvgBlocks,AvgFouls
0,2003,1102,0.250000,-3.0,-32,38,12,16,0.428571,20.821429,...,15.607143,15.035714,21.928571,6.714286,20.714286,13.107143,11.642857,6.035714,2.392857,16.571429
1,2003,1103,0.629630,-2.0,-16,33,13,14,0.481481,28.962963,...,15.481481,18.518519,25.851852,10.111111,22.703704,17.407407,13.666667,6.814815,2.555556,20.222222
2,2003,1104,4.285714,6.0,-19,28,17,11,0.607143,25.500000,...,19.821429,15.071429,21.250000,12.285714,24.785714,13.607143,12.607143,6.785714,4.178571,16.964286
3,2003,1105,-4.884615,-3.5,-42,34,7,19,0.269231,27.692308,...,18.884615,17.230769,25.076923,13.576923,27.000000,16.730769,17.730769,10.500000,3.653846,18.923077
4,2003,1106,-0.142857,-1.0,-26,26,13,15,0.464286,24.500000,...,16.107143,13.928571,20.821429,11.928571,26.250000,13.535714,15.892857,9.107143,3.785714,16.250000


Below, we add the seeds to the NCAA tournament games to the NCAA Tourney Data that contains the data with the game-level unit of observation.

In [9]:
# 1a) Load the seeds

seeds = pd.read_csv("MNCAATourneySeeds.csv")
# seeds has columns: Season, Seed (like "W01"), TeamID

# 1b) Pull out just the numeric part of the seed
seeds["SeedNum"] = seeds["Seed"].str.extract(r"(\d+)").astype(int)

# 1c) Merge for winners
seeded_tourney_games = MNCAATourneyDetailedResults_df.merge(
    seeds.rename(columns={
      "TeamID":"WTeamID",
      "Seed":"WSeed",
      "SeedNum":"WSeedNum"
    }),
    on=["Season","WTeamID"],
    how="left"
)

# 1d) Merge for losers
seeded_tourney_games = seeded_tourney_games.merge(
    seeds.rename(columns={
      "TeamID":"LTeamID",
      "Seed":"LSeed",
      "SeedNum":"LSeedNum"
    }),
    on=["Season","LTeamID"],
    how="left"
)


In [10]:
seeds.tail() # This contains seeds for 2025 as well

,Season,Seed,TeamID,SeedNum
2621,2025,Z12,1161,12
2622,2025,Z13,1213,13
2623,2025,Z14,1423,14
2624,2025,Z15,1303,15
2625,2025,Z16,1313,16


In [11]:
seeded_tourney_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1382 entries, 0 to 1381
Data columns (total 38 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Season    1382 non-null   int64 
 1   DayNum    1382 non-null   int64 
 2   WTeamID   1382 non-null   int64 
 3   WScore    1382 non-null   int64 
 4   LTeamID   1382 non-null   int64 
 5   LScore    1382 non-null   int64 
 6   WLoc      1382 non-null   object
 7   NumOT     1382 non-null   int64 
 8   WFGM      1382 non-null   int64 
 9   WFGA      1382 non-null   int64 
 10  WFGM3     1382 non-null   int64 
 11  WFGA3     1382 non-null   int64 
 12  WFTM      1382 non-null   int64 
 13  WFTA      1382 non-null   int64 
 14  WOR       1382 non-null   int64 
 15  WDR       1382 non-null   int64 
 16  WAst      1382 non-null   int64 
 17  WTO       1382 non-null   int64 
 18  WStl      1382 non-null   int64 
 19  WBlk      1382 non-null   int64 
 20  WPF       1382 non-null   int64 
 21  LFGM      1382

In [12]:
seeded_tourney_games.tail() # But we do not have tourney game results data for 2025 (we will simulate and predict)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDR,LAst,LTO,LStl,LBlk,LPF,WSeed,WSeedNum,LSeed,LSeedNum
1377,2024,146,1301,76,1181,64,N,0,28,60,...,27,11,9,4,5,23,Z11,11,Z04,4
1378,2024,146,1345,72,1397,66,N,0,24,53,...,17,17,6,8,4,25,Y01,1,Y02,2
1379,2024,152,1163,86,1104,72,N,0,31,62,...,21,9,7,2,5,15,W01,1,X04,4
1380,2024,152,1345,63,1301,50,N,0,22,55,...,22,10,11,8,3,13,Y01,1,Z11,11
1381,2024,154,1163,75,1345,60,N,0,30,62,...,19,8,9,3,3,15,W01,1,Y01,1


In [13]:
# Add Game Result column to historic tourney data
df_team_tourney_results = pd.concat(
    [
        seeded_tourney_games[
            ["Season", "WTeamID", "LTeamID", "WScore", "LScore",'WSeedNum','LSeedNum']
        ]
        .assign(GameResult="W")
        .rename(
            columns={
                "WTeamID": "TeamID",
                "LTeamID": "OppTeamID",
                "WScore": "TeamScore",
                "LScore": "OppScore",
                'WSeedNum':'SeedNum',
                'LSeedNum':'OppSeedNum'
            }
        ),
        seeded_tourney_games[
            ["Season", "LTeamID", "WTeamID", "LScore", "WScore",'WSeedNum','LSeedNum']
        ]
        .assign(GameResult="L")
        .rename(
            columns={
                "LTeamID": "TeamID",
                "WTeamID": "OppTeamID",
                "LScore": "TeamScore",
                "WScore": "OppScore",
                'LSeedNum':'SeedNum',
                'WSeedNum':'OppSeedNum'
            }
        ),
    ]
).reset_index(drop=True)

df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype(
    "int"
)


In [14]:
final_df = df_team_tourney_results.merge(
    team_season_agg[
        ['Season','TeamID','AvgScoreDiff','MedianScoreDiff','WinPercentage', 'AvgFGM', 'AvgFGA', 'AvgFGPerc', 'Avg3PM', 'Avg3PA', 'AvgFTM', 'AvgFTA', 'AvgRebounds', 'AvgDefRebounds', 'AvgAssists', 'AvgTurnovers', 'AvgSteals', 'AvgBlocks', 'AvgFouls'
]
    ],
    on=["Season", 'TeamID'],
    how="left",
).merge(
    team_season_agg[
        ['Season','TeamID','AvgScoreDiff','MedianScoreDiff','WinPercentage', 'AvgFGM', 'AvgFGA', 'AvgFGPerc', 'Avg3PM', 'Avg3PA', 'AvgFTM', 'AvgFTA', 'AvgRebounds', 'AvgDefRebounds', 'AvgAssists', 'AvgTurnovers', 'AvgSteals', 'AvgBlocks', 'AvgFouls'
]
    ].rename(
    columns={
        "TeamID": "OppTeamID",
        "AvgScoreDiff": "OppAvgScoreDiff",
        "MedianScoreDiff": "OppMedianScoreDiff",
        "WinPercentage": "OppWinPercentage",
        "AvgFGM": "OppAvgFGM",
        "AvgFGA": "OppAvgFGA",
        "AvgFGPerc": "OppAvgFGPerc",
        "Avg3PM": "OppAvg3PM",
        "Avg3PA": "OppAvg3PA",
        "AvgFTM": "OppAvgFTM",
        "AvgFTA": "OppAvgFTA",
        "AvgRebounds": "OppAvgRebounds",
        "AvgDefRebounds": "OppAvgDefRebounds",
        "AvgAssists": "OppAvgAssists",
        "AvgTurnovers": "OppAvgTurnovers",
        "AvgSteals": "OppAvgSteals",
        "AvgBlocks": "OppAvgBlocks",
        "AvgFouls": "OppAvgFouls",
    }
)
,
    on=["Season", "OppTeamID"],
)

In [15]:
print(final_df.info())
final_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Season              2764 non-null   int64  
 1   TeamID              2764 non-null   int64  
 2   OppTeamID           2764 non-null   int64  
 3   TeamScore           2764 non-null   int64  
 4   OppScore            2764 non-null   int64  
 5   SeedNum             2764 non-null   int64  
 6   OppSeedNum          2764 non-null   int64  
 7   GameResult          2764 non-null   object 
 8   Win                 2764 non-null   int64  
 9   AvgScoreDiff        2764 non-null   float64
 10  MedianScoreDiff     2764 non-null   float64
 11  WinPercentage       2764 non-null   float64
 12  AvgFGM              2764 non-null   float64
 13  AvgFGA              2764 non-null   float64
 14  AvgFGPerc           2764 non-null   float64
 15  Avg3PM              2764 non-null   float64
 16  Avg3PA

,Season,TeamID,OppTeamID,TeamScore,OppScore,SeedNum,OppSeedNum,GameResult,Win,AvgScoreDiff,...,OppAvg3PA,OppAvgFTM,OppAvgFTA,OppAvgRebounds,OppAvgDefRebounds,OppAvgAssists,OppAvgTurnovers,OppAvgSteals,OppAvgBlocks,OppAvgFouls
2759,2024,1181,1301,64,76,4,11,L,0,12.406250,...,20.916667,15.500000,20.611111,8.666667,24.694444,13.722222,9.000000,7.222222,3.722222,15.416667
2760,2024,1397,1345,66,72,2,1,L,0,11.593750,...,21.000000,16.969697,23.878788,10.424242,25.727273,18.757576,9.848485,6.454545,3.515152,14.969697
2761,2024,1104,1163,72,86,4,1,L,0,9.687500,...,23.529412,14.794118,19.941176,9.705882,25.382353,18.911765,8.941176,6.411765,5.647059,15.882353
2762,2024,1301,1345,50,63,11,1,L,0,3.666667,...,21.000000,16.969697,23.878788,10.424242,25.727273,18.757576,9.848485,6.454545,3.515152,14.969697
2763,2024,1345,1163,60,75,1,1,L,0,13.242424,...,23.529412,14.794118,19.941176,9.705882,25.382353,18.911765,8.941176,6.411765,5.647059,15.882353


Above, we have data in format we want. Now, we can start with training a model on the features, predicting and evaluating the model.

In [16]:
# feature_cols = [c for c in final_df.columns if c not in ['Season',	'TeamID','OppTeamID',	'TeamScore',	'OppScore',	'SeedNum',	'OppSeedNum',	'GameResult','Win']]
feature_cols = final_df.columns[9:26]
X = pd.DataFrame({
    f"{c}_diff": final_df[c] - final_df[f"Opp{c}"]
    for c in feature_cols
})

In [17]:
X.head()

,AvgScoreDiff_diff,MedianScoreDiff_diff,WinPercentage_diff,AvgFGM_diff,AvgFGA_diff,AvgFGPerc_diff,Avg3PM_diff,Avg3PA_diff,AvgFTM_diff,AvgFTA_diff,AvgRebounds_diff,AvgDefRebounds_diff,AvgAssists_diff,AvgTurnovers_diff,AvgSteals_diff,AvgBlocks_diff,AvgFouls_diff
0,-9.208046,-5.5,-0.151724,2.279310,5.086207,2.279310,0.178161,-0.860920,0.306897,-2.410345,2.518391,-1.464368,1.566667,-2.626437,2.978161,1.370115,-0.329885
1,10.309113,7.0,0.237685,4.806650,9.264778,4.806650,1.068966,2.549261,3.961823,5.004926,2.336207,1.665025,2.417488,1.726601,0.733990,0.285714,2.336207
2,-1.896552,-2.0,-0.172414,1.896552,-3.620690,1.896552,-2.379310,-6.206897,2.655172,2.965517,-1.068966,-1.827586,-0.517241,0.000000,-1.310345,0.344828,0.965517
3,-8.805643,-6.0,-0.085684,-1.215256,-4.660397,-1.215256,-1.102403,-3.196447,5.666667,5.858934,-0.629049,0.221526,-0.949843,4.094044,-0.890282,-0.446186,2.904911
4,0.324138,1.0,0.124138,2.121839,4.675862,2.121839,-1.319540,-3.118391,-2.681609,-1.797701,1.205747,2.331034,0.990805,0.775862,-1.541379,-0.701149,-0.850575


In [18]:
# Simple training on previous seasons and predicting 2024
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, confusion_matrix
from xgboost import XGBClassifier

y = final_df["Win"]

# 4) Drop rows with any NaNs
mask = X.notna().all(axis=1)
X, y = X[mask], y[mask]

# 5) Train/test split by season
seasons = final_df.loc[mask, "Season"] # this is a df of just the Seasons column of rows that have no NaNs.
train_mask = seasons <= 2023 # Training on seasons prior to 2023.
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

# 6) Fit & evaluate
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

p_test = model.predict_proba(X_test)[:,1]
print("Log loss:   ", log_loss(y_test, p_test))
print("Accuracy:   ", accuracy_score(y_test, p_test>0.5))
print("ROC AUC:    ", roc_auc_score(y_test, p_test))
print("Confusion:\n", confusion_matrix(y_test, p_test>0.5))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:25:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Log loss:    0.7239459201372319
Accuracy:    0.6194029850746269
ROC AUC:     0.6981510358654489
Confusion:
 [[41 26]
 [25 42]]


In [19]:
# Training multiple models (one per season left out) to evaluate performance on unseen seasons
from sklearn.model_selection import GroupKFold
import xgboost as xgb


groups = final_df["Season"]
seasons = final_df["Season"].unique()

# Setup cross-validation
gkf = GroupKFold(n_splits=final_df["Season"].nunique())
cv_results = []
models = []

season_idx = 0
for train_index, test_index in gkf.split(X, y, groups):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # Prepare the model
  model = xgb.XGBRegressor(eval_metric="logloss", n_estimators=1_000, learning_rate=0.001,)
  holdout_season = seasons[season_idx]
  print(f"Holdout Season: {holdout_season}")

  # Train the model
  model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

  # Predict on the test set
  y_pred = model.predict(X_test)
  score_ll = log_loss(y_test, y_pred)
  y_pred = y_pred > 0.5
  # Evaluate the model
  accuracy = accuracy_score(y_test, y_pred)
  cv_results.append(accuracy)
  season_idx += 1
  print(f"Season {holdout_season}: {accuracy} {score_ll}")
  models.append(model)

# Print the average accuracy across all folds
print("Average CV Accuracy:", np.mean(cv_results))

Holdout Season: 2003
[0]	validation_0-logloss:0.69289
[100]	validation_0-logloss:0.67302
[200]	validation_0-logloss:0.65780
[300]	validation_0-logloss:0.64516
[400]	validation_0-logloss:0.63567
[500]	validation_0-logloss:0.62855
[600]	validation_0-logloss:0.62262
[700]	validation_0-logloss:0.61760
[800]	validation_0-logloss:0.61241
[900]	validation_0-logloss:0.60944
[999]	validation_0-logloss:0.60644
Season 2003: 0.6865671641791045 0.60643911540291
Holdout Season: 2004
[0]	validation_0-logloss:0.69298
[100]	validation_0-logloss:0.67853
[200]	validation_0-logloss:0.66742
[300]	validation_0-logloss:0.65943
[400]	validation_0-logloss:0.65181
[500]	validation_0-logloss:0.64574
[600]	validation_0-logloss:0.64112
[700]	validation_0-logloss:0.63651
[800]	validation_0-logloss:0.63286
[900]	validation_0-logloss:0.63028
[999]	validation_0-logloss:0.62757
Season 2004: 0.6343283582089553 0.6275729776130302
Holdout Season: 2005
[0]	validation_0-logloss:0.69305
[100]	validation_0-logloss:0.68236
[20

In [20]:
final_df["BaselinePred"] = (
    final_df["SeedNum"]
    < final_df["OppSeedNum"]
)

final_df.loc[
    final_df["SeedNum"]
    == final_df["OppSeedNum"],
    "BaselinePred",
] = (
    final_df["WinPercentage"]
    > final_df["OppWinPercentage"]
)

In [21]:
cv_scores_baseline = []
for season in final_df["Season"].unique():
    pred = final_df.query("Season == @season")[
        "BaselinePred"
    ].astype("int")
    y = final_df.query("Season == @season")["Win"]
    score = accuracy_score(y, pred)
    score_ll = log_loss(y, pred)
    cv_scores_baseline.append(score)
    print(f"Holdout season {season} - Accuracy {score:0.4f} Log Loss {score_ll:0.4f}")

print(f"Baseline accuracy {np.mean(cv_scores_baseline):0.4f}")

Holdout season 2003 - Accuracy 0.6562 Log Loss 12.3900
Holdout season 2004 - Accuracy 0.7344 Log Loss 9.5741
Holdout season 2005 - Accuracy 0.6719 Log Loss 11.8268
Holdout season 2006 - Accuracy 0.6719 Log Loss 11.8268
Holdout season 2007 - Accuracy 0.7969 Log Loss 7.3214
Holdout season 2008 - Accuracy 0.7656 Log Loss 8.4477
Holdout season 2009 - Accuracy 0.7344 Log Loss 9.5741
Holdout season 2010 - Accuracy 0.6719 Log Loss 11.8268
Holdout season 2011 - Accuracy 0.6866 Log Loss 11.2973
Holdout season 2012 - Accuracy 0.7015 Log Loss 10.7593
Holdout season 2013 - Accuracy 0.6716 Log Loss 11.8352
Holdout season 2014 - Accuracy 0.6343 Log Loss 13.1801
Holdout season 2015 - Accuracy 0.7463 Log Loss 9.1454
Holdout season 2016 - Accuracy 0.6716 Log Loss 11.8352
Holdout season 2017 - Accuracy 0.7463 Log Loss 9.1454
Holdout season 2018 - Accuracy 0.6716 Log Loss 11.8352
Holdout season 2019 - Accuracy 0.6866 Log Loss 11.2973
Holdout season 2021 - Accuracy 0.6970 Log Loss 10.9223
Holdout season 2

On average, "Chalk" baseline outperforms our XG-Boost Model right now. We could play with features to try to improve this.

# Predicting 2025

In [22]:
seeds_2025 = seeds[seeds['Season']==2025]


In [23]:
seeds_2025

,Season,Seed,TeamID,SeedNum
2558,2025,W01,1181,1
2559,2025,W02,1104,2
2560,2025,W03,1458,3
2561,2025,W04,1112,4
2562,2025,W05,1332,5
...,...,...,...,...
2621,2025,Z12,1161,12
2622,2025,Z13,1213,13
2623,2025,Z14,1423,14
2624,2025,Z15,1303,15


In [24]:
matchups = seeds.merge(seeds, on="Season").rename(columns={
    "TeamID_x": "TeamID",  # Original team's TeamID
    "Seed_x": "Seed",       # Original team's Seed
    "SeedNum_x": "SeedNum", # Original team's SeedNum
    "TeamID_y": "OppTeamID", # Opponent's TeamID
    "Seed_y": "OppSeed",     # Opponent's Seed
    "SeedNum_y": "OppSeedNum" # Opponent's SeedNum
})

# Remove matchups where the same team would play itself
matchups = matchups[matchups["TeamID"] != matchups["OppTeamID"]]

# Optional: reset index for cleanliness
matchups = matchups.reset_index(drop=True)

# View the matchups
matchups.head()


,Season,Seed,TeamID,SeedNum,OppSeed,OppTeamID,OppSeedNum
0,1985,W01,1207,1,W02,1210,2
1,1985,W01,1207,1,W03,1228,3
2,1985,W01,1207,1,W04,1260,4
3,1985,W01,1207,1,W05,1374,5
4,1985,W01,1207,1,W06,1208,6


In [25]:
# Merge stats for TeamID
final_games = matchups.merge(
    team_season_agg[
        ['Season','TeamID','AvgScoreDiff','MedianScoreDiff','WinPercentage', 'AvgFGM', 'AvgFGA', 'AvgFGPerc', 'Avg3PM', 'Avg3PA', 'AvgFTM', 'AvgFTA', 'AvgRebounds', 'AvgDefRebounds', 'AvgAssists', 'AvgTurnovers', 'AvgSteals', 'AvgBlocks', 'AvgFouls'
]
    ],
    on=["Season", 'TeamID'],
    how="left",
).merge(
    team_season_agg[
        ['Season','TeamID','AvgScoreDiff','MedianScoreDiff','WinPercentage', 'AvgFGM', 'AvgFGA', 'AvgFGPerc', 'Avg3PM', 'Avg3PA', 'AvgFTM', 'AvgFTA', 'AvgRebounds', 'AvgDefRebounds', 'AvgAssists', 'AvgTurnovers', 'AvgSteals', 'AvgBlocks', 'AvgFouls'
]
    ].rename(
    columns={
        "TeamID": "OppTeamID",
        "AvgScoreDiff": "OppAvgScoreDiff",
        "MedianScoreDiff": "OppMedianScoreDiff",
        "WinPercentage": "OppWinPercentage",
        "AvgFGM": "OppAvgFGM",
        "AvgFGA": "OppAvgFGA",
        "AvgFGPerc": "OppAvgFGPerc",
        "Avg3PM": "OppAvg3PM",
        "Avg3PA": "OppAvg3PA",
        "AvgFTM": "OppAvgFTM",
        "AvgFTA": "OppAvgFTA",
        "AvgRebounds": "OppAvgRebounds",
        "AvgDefRebounds": "OppAvgDefRebounds",
        "AvgAssists": "OppAvgAssists",
        "AvgTurnovers": "OppAvgTurnovers",
        "AvgSteals": "OppAvgSteals",
        "AvgBlocks": "OppAvgBlocks",
        "AvgFouls": "OppAvgFouls",
    }
)
,
    on=["Season", "OppTeamID"],
)


# Now final_games has both teams' stats in one row
print(final_games.columns)
final_games.head()



Index(['Season', 'Seed', 'TeamID', 'SeedNum', 'OppSeed', 'OppTeamID',
       'OppSeedNum', 'AvgScoreDiff', 'MedianScoreDiff', 'WinPercentage',
       'AvgFGM', 'AvgFGA', 'AvgFGPerc', 'Avg3PM', 'Avg3PA', 'AvgFTM', 'AvgFTA',
       'AvgRebounds', 'AvgDefRebounds', 'AvgAssists', 'AvgTurnovers',
       'AvgSteals', 'AvgBlocks', 'AvgFouls', 'OppAvgScoreDiff',
       'OppMedianScoreDiff', 'OppWinPercentage', 'OppAvgFGM', 'OppAvgFGA',
       'OppAvgFGPerc', 'OppAvg3PM', 'OppAvg3PA', 'OppAvgFTM', 'OppAvgFTA',
       'OppAvgRebounds', 'OppAvgDefRebounds', 'OppAvgAssists',
       'OppAvgTurnovers', 'OppAvgSteals', 'OppAvgBlocks', 'OppAvgFouls'],
      dtype='object')


,Season,Seed,TeamID,SeedNum,OppSeed,OppTeamID,OppSeedNum,AvgScoreDiff,MedianScoreDiff,WinPercentage,...,OppAvg3PA,OppAvgFTM,OppAvgFTA,OppAvgRebounds,OppAvgDefRebounds,OppAvgAssists,OppAvgTurnovers,OppAvgSteals,OppAvgBlocks,OppAvgFouls
0,2003,W01,1328,1,W02,1448,2,11.0,8.5,0.8,...,17.724138,20.620690,27.241379,14.034483,27.241379,15.586207,15.034483,6.448276,4.344828,18.000000
1,2003,W01,1328,1,W03,1393,3,11.0,8.5,0.8,...,16.448276,16.103448,23.310345,13.896552,27.413793,16.241379,13.344828,8.413793,6.793103,17.137931
2,2003,W01,1328,1,W04,1257,4,11.0,8.5,0.8,...,21.800000,18.900000,26.533333,13.466667,25.733333,16.100000,13.500000,7.233333,4.266667,22.033333
3,2003,W01,1328,1,W05,1280,5,11.0,8.5,0.8,...,15.700000,13.100000,19.533333,12.166667,25.166667,14.666667,15.200000,8.766667,3.900000,15.300000
4,2003,W01,1328,1,W06,1329,6,11.0,8.5,0.8,...,14.379310,15.068966,23.275862,12.758621,24.931034,13.137931,14.655172,8.793103,5.551724,19.034483


In [26]:
feature_cols = final_df.columns[9:26]
# List to store the new diff column names
diff_columns = []

# Assuming feature_cols contains the list of column names you're working with
for c in feature_cols:
    diff_column_name = f"{c}_diff"
    final_games[diff_column_name] = final_games[c] - final_games[f"Opp{c}"]
    diff_columns.append(diff_column_name)

# Now final_games has the new diff columns, and diff_columns holds the list of their names
print(diff_columns)  # This will show the list of the new diff columns



['AvgScoreDiff_diff', 'MedianScoreDiff_diff', 'WinPercentage_diff', 'AvgFGM_diff', 'AvgFGA_diff', 'AvgFGPerc_diff', 'Avg3PM_diff', 'Avg3PA_diff', 'AvgFTM_diff', 'AvgFTA_diff', 'AvgRebounds_diff', 'AvgDefRebounds_diff', 'AvgAssists_diff', 'AvgTurnovers_diff', 'AvgSteals_diff', 'AvgBlocks_diff', 'AvgFouls_diff']


In [27]:
for i, model in enumerate(models):
    final_games[f"pred_model{i}"] = model.predict(final_games[diff_columns])

In [28]:
final_games["Pred"] = final_games[
    [f for f in final_games.columns if "model" in f]
].mean(axis=1)

final_games["ID"] = (
    final_games["Season"].astype("str")
    + "_"
    + final_games["TeamID"].astype("str")
    + "_"
    + final_games["OppTeamID"].astype("str")
)

preds = final_games.copy()

In [29]:
preds

,Season,Seed,TeamID,SeedNum,OppSeed,OppTeamID,OppSeedNum,AvgScoreDiff,MedianScoreDiff,WinPercentage,...,pred_model13,pred_model14,pred_model15,pred_model16,pred_model17,pred_model18,pred_model19,pred_model20,Pred,ID
0,2003,W01,1328,1,W02,1448,2,11.000000,8.5,0.800000,...,0.547803,0.504638,0.486813,0.515888,0.605188,0.530514,0.559238,0.522470,0.541320,2003_1328_1448
1,2003,W01,1328,1,W03,1393,3,11.000000,8.5,0.800000,...,0.380065,0.428815,0.384415,0.376718,0.396238,0.340244,0.396546,0.420464,0.402188,2003_1328_1393
2,2003,W01,1328,1,W04,1257,4,11.000000,8.5,0.800000,...,0.458512,0.460979,0.492932,0.461976,0.449498,0.447613,0.416995,0.437385,0.464960,2003_1328_1257
3,2003,W01,1328,1,W05,1280,5,11.000000,8.5,0.800000,...,0.525306,0.445598,0.495661,0.441531,0.535053,0.500380,0.551402,0.550499,0.497815,2003_1328_1280
4,2003,W01,1328,1,W06,1329,6,11.000000,8.5,0.800000,...,0.604189,0.572207,0.637626,0.514500,0.614549,0.551713,0.560124,0.622471,0.565457,2003_1328_1329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97059,2025,Z16,1313,16,Z11,1179,11,3.434783,6.0,0.652174,...,0.389236,0.394209,0.411381,0.396431,0.356109,0.355534,0.357749,0.432834,0.383743,2025_1313_1179
97060,2025,Z16,1313,16,Z12,1161,12,3.434783,6.0,0.652174,...,0.476579,0.509445,0.503983,0.541615,0.499938,0.503654,0.571376,0.523812,0.504578,2025_1313_1161
97061,2025,Z16,1313,16,Z13,1213,13,3.434783,6.0,0.652174,...,0.445301,0.424411,0.431212,0.439200,0.430953,0.488242,0.456049,0.469035,0.434085,2025_1313_1213
97062,2025,Z16,1313,16,Z14,1423,14,3.434783,6.0,0.652174,...,0.544910,0.560477,0.527672,0.590261,0.544143,0.596573,0.574811,0.555612,0.556760,2025_1313_1423


In [30]:
from tqdm import tqdm

# Load and filter data
round_slots = pd.read_csv(
    "MNCAATourneySlots.csv"
)
round_slots = round_slots[round_slots["Season"] == 2025]
round_slots = round_slots[
round_slots["Slot"].str.contains("R")]  # Filter out First Four

seeds = seeds_2025

preds["ID"] = preds["ID"].str.split("_")


In [31]:
round_slots

,Season,Slot,StrongSeed,WeakSeed
2519,2025,R1W1,W01,W16
2520,2025,R1W2,W02,W15
2521,2025,R1W3,W03,W14
2522,2025,R1W4,W04,W13
2523,2025,R1W5,W05,W12
...,...,...,...,...
2577,2025,R4Y1,R3Y1,R3Y2
2578,2025,R4Z1,R3Z1,R3Z2
2579,2025,R5WX,R4W1,R4X1
2580,2025,R5YZ,R4Y1,R4Z1


In [32]:
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["Pred"]):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    # Preprocess play-in games (like W16a vs W16b)
    play_in_keys = [k for k in seeds.keys() if k.endswith("a") or k.endswith("b")]
    play_in_bases = set(k[:-1] for k in play_in_keys)

    for base in play_in_bases:
        a_key, b_key = base + "a", base + "b"
        if a_key in seeds and b_key in seeds:
            team1, team2 = seeds[a_key], seeds[b_key]
            proba = probas[str(team1)][str(team2)]
            winner = np.random.choice([team1, team2], p=[proba, 1 - proba])
            seeds[base] = winner  # Save under the non-a/b version

    winners = []
    slots = []

    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning
        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1
result_m = run_simulation(
    brackets=n_brackets, seeds=seeds, preds=preds, round_slots=round_slots, sim=False
)


100%|██████████| 1/1 [00:00<00:00, 542.81it/s]


In [33]:
result_m

,Bracket,Slot,Team
0,1,R1W1,W01
1,1,R1W2,W02
2,1,R1W3,W03
3,1,R1W4,W04
4,1,R1W5,W12
...,...,...,...
58,1,R4Y1,Y01
59,1,R4Z1,Z04
60,1,R5WX,X01
61,1,R5YZ,Z04


In [34]:
predictions_with_names = result_m.rename(columns={"Team": "Seed"}).merge(
    seeds, on=["Seed"], how="left"
)

predictions_with_names = predictions_with_names.merge(
    MTeams_df[[ "TeamID", "TeamName"]], how="left"
)
print(predictions_with_names.to_string())

    Bracket  Slot Seed  Season  TeamID  SeedNum        TeamName
0         1  R1W1  W01    2025    1181        1            Duke
1         1  R1W2  W02    2025    1104        2         Alabama
2         1  R1W3  W03    2025    1458        3       Wisconsin
3         1  R1W4  W04    2025    1112        4         Arizona
4         1  R1W5  W12    2025    1251       12         Liberty
5         1  R1W6  W06    2025    1140        6             BYU
6         1  R1W7  W07    2025    1388        7    St Mary's CA
7         1  R1W8  W08    2025    1280        8  Mississippi St
8         1  R1X1  X01    2025    1222        1         Houston
9         1  R1X2  X02    2025    1397        2       Tennessee
10        1  R1X3  X14    2025    1407       14            Troy
11        1  R1X4  X04    2025    1345        4          Purdue
12        1  R1X5  X05    2025    1155        5         Clemson
13        1  R1X6  X06    2025    1228        6        Illinois
14        1  R1X7  X07    2025    1417  

In [35]:
# Maryland vs. Florida for ex.
preds[(preds['Season']==2025)&(preds['TeamID']==1196)&(preds['OppTeamID']==1268)]


,Season,Seed,TeamID,SeedNum,OppSeed,OppTeamID,OppSeedNum,AvgScoreDiff,MedianScoreDiff,WinPercentage,...,pred_model13,pred_model14,pred_model15,pred_model16,pred_model17,pred_model18,pred_model19,pred_model20,Pred,ID
96046,2025,Z01,1196,1,Z04,1268,4,17.423077,17.5,0.884615,...,0.450643,0.407451,0.446908,0.450287,0.474533,0.445673,0.502249,0.454385,0.45232,"[2025, 1196, 1268]"


In [36]:
# predictions_with_names.to_csv("/content/predictions_with_names.csv")


- I filled in results in a csv called predictions_with_outcomes
- This is now predictions_with_names.csv in our data folder
- I also filled out chalk predictions

In [37]:
pwd

'/content/DS3001-Project/data'

In [43]:
df = pd.read_csv("predictions_with_outcomes.csv")
df["Correct"] = df["TeamName"] == df["CorrectTeamName"]
accuracy = df["Correct"].mean()
print(accuracy)
df["Chalk_Correct"] = df["Chalk"] == df["CorrectTeamName"]
chalk_accuracy = df["Chalk_Correct"].mean()
print(chalk_accuracy)

0.6349206349206349
0.7619047619047619


- Chalk greatly outperformed our model this season
- Important to note that March Madness was particularly chalky this season
- We could play around more with features for example to try to improve our model in the future